## Ngram Model (updated)

Due to the perfect result which is trained on all the dataset, this code is trying to separate the train and test dataset to evaluate the result. 

## input dataset

In [1]:
import pickle 

In [2]:
with open('/Users/apple/Documents/GitHub/Argument-Scoring-System/pretrained_model/labeled_essay_dics.pickle', 'rb') as handle:
    label_sents = pickle.load(handle)

In [3]:
# compute and store and the {sents: label} into one dic
all_label_sents = {}
for key in label_sents.keys():
    this_essay = label_sents[key]
    for s in this_essay.keys():
        all_label_sents[s] = this_essay[s]

## clear data

In [4]:
import re
import itertools
from collections import Counter

In [5]:
"""
Original taken from https://github.com/dennybritz/cnn-text-classification-tf
"""


def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [6]:
def clear_split_str(s):
    strip_s = s.strip()
    clear_s = clean_str(strip_s)
    s_text = clear_s.split(" ")
    return s_text

In [7]:
X = []
y = []
for key in all_label_sents.keys():
    this_value = all_label_sents[key]
    clear_key = clear_split_str(key)
    X.append(clear_key)
    y.append(this_value)

## train test split

In [8]:
from sklearn.cross_validation import train_test_split

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                   y,
                                                   test_size = 0.1,
#                                                    shuffle = True
                                                   )

## separate by class 

In [10]:
claim_list = []
premise_list = []
major_claim_list = []
other_list = []

for i in range(0, len(X_train)):
    this_value = y_train[i]
    clear_key = X_train[i]
    if this_value == "Premise":
        premise_list.append(clear_key)
    elif this_value == "Claim":
        claim_list.append(clear_key)
    elif this_value == "MajorClaim":
        major_claim_list.append(clear_key)
    else:
        other_list.append(clear_key)

In [11]:
# print (claim_list[1:10])
print (len(premise_list) + len(claim_list) + len(major_claim_list) + len(other_list) + len(y_test))
# print ()

7077


## ngram

In [12]:
import collections
from collections import defaultdict

In [13]:
# a bug here, the method of computing pro is wrong 
def ngram(sent_list, n):
    word_num = defaultdict(float)
    total_num = 0
    for sent in sent_list:
        for i in range(len(sent)-n+1):
            this_key = ' '.join(sent[i:i+n])
            word_num[this_key] = word_num.get(this_key,0) + 1
            total_num = total_num + 1
    word_pro = {}
    for key in word_num:
        word_pro[key] = word_num[key]/float(total_num)
    return word_pro

In [14]:
unigram_claim = ngram(claim_list,1)
unigram_premise = ngram(premise_list,1)
unigram_major_claim = ngram(major_claim_list,1)
unigram_others = ngram(other_list,1)

In [15]:
bigram_claim = ngram(claim_list,2)
bigram_premise = ngram(premise_list,2)
bigram_major_claim = ngram(major_claim_list,2)
bigram_others = ngram(other_list,2)

In [16]:
trigram_claim = ngram(claim_list,3)
trigram_premise = ngram(premise_list,3)
trigram_major_claim = ngram(major_claim_list,3)
trigram_others = ngram(other_list,3)

In [17]:
fourgram_claim = ngram(claim_list,4)
fourgram_premise = ngram(premise_list,4)
fourgram_major_claim = ngram(major_claim_list,4)
fourgram_others = ngram(other_list,4)

## new ngram

using the right equation to compute the ngram

In [18]:
### bug here

def ngram1_4(sent_list):
    '''
    computer the 1 to 4 gram, and return the list
    sent_list: list of sents
    return: a dic includes one to four grams 
    '''
    # unigram
    word_num = defaultdict(float)
    total_num = 0
    for sent in sent_list:
        for i in range(len(sent)-1+1):
            this_key = ' '.join(sent[i:i+1])
            word_num[this_key] = word_num.get(this_key,0) + 1
            total_num = total_num + 1
    word_pro = {}
    for key in word_num:
        word_pro[key] = word_num[key]/float(total_num)
        
    # bigram
    bi_num = defaultdict(float)
    for sent in sent_list:
        for i in range(len(sent)-2+1):
            this_key = ' '.join(sent[i:i+2])
            bi_num[this_key] = bi_num.get(this_key,0) + 1
    bi_pro = {}
    for key in bi_num:
        pre_key = key.split()[0]
        bi_pro[key] = bi_num[key]/float(word_num[pre_key])
    
    # trigram
    tri_num = defaultdict(float)
    for sent in sent_list:
        for i in range(len(sent)-3+1):
            this_key = ' '.join(sent[i:i+3])
            bi_num[this_key] = bi_num.get(this_key,0) + 1
    tri_pro = {}
    for key in bi_num:
        pre_key = key.split()[0:2]
        tri_pro[key] = tri_num[key]/float(word_num[pre_key])
    
    # fourgram
    
    four_num = defaultdict(float)
    for sent in sent_list:
        for i in range(len(sent)-4+1):
            this_key = ' '.join(sent[i:i+4])
            bi_num[this_key] = bi_num.get(this_key,0) + 1
    bi_pro = {}
    for key in bi_num:
        pre_key = key.split()[0:2]
        bi_pro[key] = bi_num[key]/float(word_num[pre_key])
    
    
    
    gram_dic = {}
    gram_dic[1] = word_pro
    gram_dic[2] = bi_pro
    
    return gram_dic

## build classifier

In [19]:
from math import log,exp

In [20]:
def get_grams(sent, n):
    gram_list = []
    for i in range(len(sent)-n+1):
        gram_list.append(' '.join(sent[i:i+n]))
    return gram_list

In [21]:
import sys
epslon = sys.float_info.epsilon

In [22]:
# Bug here!!!!
def sent_prob(keys, pro_dic):
    result = 0.0
    for key in keys:
        result = result + (log(pro_dic.get(key, epslon)))
    return result

In [23]:
log(epslon)

-36.04365338911715

In [24]:
log(0.00001)

-11.512925464970229

In [25]:
def sum_gram(sent):
    claim_sum = 0.0
    premise_sum = 0.0
    major_claim_sum = 0.0
    others_sum = 0.0
    
    p1 = 0.1
    p2 = 0.2
    p3 = 0.3
    p4 = 0.4
    
    uni_key = get_grams(sent, 1)
    claim_sum = claim_sum + p1 * sent_prob(uni_key, unigram_claim)
    premise_sum = premise_sum + p1 * sent_prob(uni_key, unigram_premise)
    major_claim_sum = major_claim_sum + p1 * sent_prob(uni_key, unigram_major_claim)
    others_sum = others_sum + p1 * sent_prob(uni_key, unigram_others)
    
    bi_key = get_grams(sent, 2)
    claim_sum = claim_sum + p2 * sent_prob(bi_key, bigram_claim)
    premise_sum = premise_sum + p2 * sent_prob(bi_key, bigram_premise)
    major_claim_sum = major_claim_sum + p2 * sent_prob(bi_key, bigram_major_claim)
    others_sum = others_sum + p2 * sent_prob(bi_key, bigram_others)   
    
    tri_key = get_grams(sent, 3)
    claim_sum = claim_sum + p3 * sent_prob(tri_key, trigram_claim)
    premise_sum = premise_sum + p3 * sent_prob(tri_key, trigram_premise)
    major_claim_sum = major_claim_sum + p3 * sent_prob(tri_key, trigram_major_claim)
    others_sum = others_sum + p3 * sent_prob(tri_key, trigram_others)      
    
    four_key = get_grams(sent, 4)
    claim_sum = claim_sum + p4 * sent_prob(four_key, fourgram_claim)
    premise_sum = premise_sum + p4 * sent_prob(four_key, fourgram_premise)
    major_claim_sum = major_claim_sum + p4 * sent_prob(four_key, fourgram_major_claim)
    others_sum = others_sum + p4 * sent_prob(four_key, fourgram_others) 
    
    pro_dic = {}
    pro_dic['Claim'] = claim_sum
    pro_dic['Premise'] = premise_sum
    pro_dic['MajorClaim'] = major_claim_sum
    pro_dic['Empty'] = others_sum
    return pro_dic

In [26]:
def max_class(values):
    key = max(values, key=lambda i: values[i])
    return key

## predict and error analyze

In [27]:
# claim_list = []
# premise_list = []
# major_claim_list = []
# other_list = []

In [28]:
y_pred = []
for item in X_test:
    this_gram = sum_gram(item)
    this_value = max_class(sum_gram(item))
    y_pred.append(this_value)

In [29]:
from sklearn.metrics import accuracy_score, f1_score

In [30]:
accuracy = accuracy_score(y_test,y_pred)

In [31]:
print (accuracy)

0.567796610169
